In [11]:
import qiskit
from qiskit import transpile
import numpy as np
import pandas as pd
import yfinance as yf
import sys
import os
os.environ["Q_ALCHEMY_API_KEY"] = "JnvkpMCsyr4nB9nHcwa6CbxqhtZXyF1b"
sys.path.append('..')
from q_alchemy.qiskit import QAlchemyInitialize
 

In [12]:
tickers = ['TSLA', 'GOOGL', 'AMZN', 'AAPL', 'META']
start_date = '2021-01-01'
end_date = '2022-01-01'

data = {}

for ticker in tickers:
    data[ticker] =  yf.download(ticker, start=start_date, end=end_date)

print(data)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
{'TSLA':                   Open        High         Low       Close   Adj Close  \
Date                                                                     
2021-01-04  239.820007  248.163330  239.063339  243.256668  243.256668   
2021-01-05  241.220001  246.946671  239.733337  245.036667  245.036667   
2021-01-06  252.830002  258.000000  249.699997  251.993332  251.993332   
2021-01-07  259.209991  272.329987  258.399994  272.013336  272.013336   
2021-01-08  285.333344  294.829987  279.463318  293.339996  293.339996   
...                ...         ...         ...         ...         ...   
2021-12-27  357.890015  372.333344  356.906677  364.64

In [14]:
for ticker in tickers:
    data[ticker]['Daily Return'] =  data[ticker]['Adj Close'].pct_change()
    data[ticker] = data[ticker].dropna()

In [15]:
expected_returns = {}
std_deviation = {}

for ticker in tickers:
    expected_returns[ticker] = data[ticker]['Daily Return'].mean()
    std_deviation[ticker] = data[ticker]['Daily Return'].std()
    
print(expected_returns)
print(std_deviation)

{'TSLA': 0.002061082797253658, 'GOOGL': 0.0021812108248734545, 'AMZN': 0.00029514969890108493, 'AAPL': 0.0014094180101045589, 'META': 0.0010642527096604967}
{'TSLA': 0.03450369927285423, 'GOOGL': 0.015311596310057506, 'AMZN': 0.01514458679788764, 'AAPL': 0.01575826098131135, 'META': 0.018632767720898034}


In [33]:
y_raw = [list(data[ticker]['Daily Return'].values) for ticker in tickers]
y_new = list(map(list, zip(*y_raw)))
y_new = np.array(y_new)

t = len(y_new)
n = len(y_new[0])

R = [expected_returns[ticker] for ticker in tickers]
R = np.array(R)

cov_matrix = np.zeros((n, n))

for i in range(t):
    cov_matrix += np.outer((y_new[i]-R), (y_new[i]-R))
cov_matrix = cov_matrix/(t-1)
print(cov_matrix)

[[0.00119051 0.00015751 0.00017421 0.00024896 0.00020076]
 [0.00015751 0.00023444 0.000136   0.0001324  0.00016547]
 [0.00017421 0.000136   0.00022936 0.00014091 0.00014446]
 [0.00024896 0.0001324  0.00014091 0.00024832 0.00014752]
 [0.00020076 0.00016547 0.00014446 0.00014752 0.00034718]]


In [43]:
from datetime import datetime

yesterday = '2023-12-20'
today = '2023-12-21'

today_price = {}

for ticker in tickers:
    data = yf.download(ticker, start=yesterday, end=today)
    today_price[ticker] = float(data['Open'].values)

print(today_price)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
{'TSLA': array([256.41000366]), 'GOOGL': array([138.97000122]), 'AMZN': array([152.8999939]), 'AAPL': array([196.8999939]), 'META': array([348.6499939])}
